In [53]:
!pip install pydicom

    100% |████████████████████████████████| 7.0MB 8.6MB/s eta 0:00:01


In [62]:
import glob
from pathlib import Path
import numpy as np
import torch
from torch import nn
import pandas as pd
import os
from collections import Counter
import pydicom
import matplotlib.pyplot as plt

In [37]:
testfiles = [x for x in list(Path('./').iterdir()) if '.ipynb' not in str(x) and 'TEST' in str(x)]
trainfiles = [x for x in list(Path('./').iterdir()) if '.ipynb' not in str(x) and 'TRAIN' in str(x)]

In [40]:
total_test_files = []
for test_file in testfiles:
    sub_files = glob.glob(str(test_file/'*'/'*'/'*'))
    total_test_files.extend([Path(x) for x in sub_files])
total_train_files = []
for test_file in trainfiles:
    sub_files = glob.glob(str(test_file/'*'/'*'/'*'))
    total_train_files.extend([Path(x) for x in sub_files])
    

In [41]:
total_test_files[:2]

[PosixPath('RTMAC-TEST-012/09-02-1998-ResearchHead and neck-41417/3-T2 TSE-82622/000119.dcm'),
 PosixPath('RTMAC-TEST-012/09-02-1998-ResearchHead and neck-41417/3-T2 TSE-82622/000118.dcm')]

In [81]:
def read_structure(structure):
    contours = []
    for i in range(len(structure.ROIContourSequence)):
        contour = {}
        contour['color'] = structure.ROIContourSequence[i].ROIDisplayColor

        contour['name'] = structure.StructureSetROISequence[i].ROIName

        contour['contours'] = [np.array(s.ContourData,dtype='float') for s in structure.ROIContourSequence[i].ContourSequence]
    contours.append(contour)
    return contours

def get_mask(contours, slices):
    z = [s.ImagePositionPatient[2] for s in slices]
    pos_r = slices[0].ImagePositionPatient[1]
    spacing_r = slices[0].PixelSpacing[1]
    pos_c = slices[0].ImagePositionPatient[0]
    spacing_c = slices[0].PixelSpacing[0]

    label = np.zeros_like(image, dtype=np.uint8)
    for con in contours:
        num = int(con['number'])
    for c in con['contours']:
        nodes = np.array(c).reshape((-1, 3))
        assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
        z_index = z.index(nodes[0, 2])
        r = (nodes[:, 1] - pos_r) / spacing_r
        c = (nodes[:, 0] - pos_c) / spacing_c
        rr, cc = polygon(r, c)
        label[rr, cc, z_index] = num

    colors = tuple(np.array([con['color'] for con in contours]) / 255.0)
    return label, colors


train_patients = [f.parts[-1] for f in total_train_files]
test_patients = [f.parts[-1] for f in total_test_files]

train_counts = Counter(train_patients)
test_counts = Counter(test_patients)

patient = total_train_files[0] # Just get the first patient for demo
for patient_file in total_train_files[0:1]:
    import pydicom.uid
    patient_id = patient_file.parts[-1]
    
    structure = pydicom.read_file(str(patient_file),force=True)
    x = read_structure(structure)
    contours = x['contours']
    print(x)
# Plot to check slices, for example 50 to 59
plt.figure(figsize=(15, 15))
for i in range(9):
    print(x[0]['contours'].shape)
    plt.subplot(3, 3, i + 1)
    plt.imshow(x[0]['contours'], cmap="gray")
    plt.contour(label[..., i + 50], levels=[0.5, 1.5, 2.5, 3.5, 4.5], colors=colors)
    plt.axis('off')

[{'color': ['122', '0', '0'], 'name': 'Parotid_R', 'contours': [array([-61.56012, -90.84505,  41.21004, -60.80606, -92.34753,  41.21004,
       -60.80621, -94.34753,  41.21004, -61.0565 , -94.84753,  41.21004,
       -61.06012, -95.35447,  41.21004, -61.55318, -95.84753,  41.21004,
       -61.55683, -96.34753,  41.21004, -62.06012, -96.85078,  41.21004,
       -62.56012, -96.85297,  41.21004, -63.05469, -97.34753,  41.21004,
       -64.06012, -97.29284,  41.21004, -64.56012, -96.94168,  41.21004,
       -65.06012, -96.70027,  41.21004, -65.56012, -95.9636 ,  41.21004,
       -66.06012, -95.73792,  41.21004, -66.56273, -95.34753,  41.21004,
       -66.79533, -94.84753,  41.21004, -69.06287, -90.34753,  41.21004,
       -69.2942 , -89.84753,  41.21004, -69.31322, -89.34753,  41.21004,
       -69.79378, -88.34753,  41.21004, -69.83028, -85.84753,  41.21004,
       -69.87395, -85.34753,  41.21004, -69.56874, -84.84753,  41.21004,
       -69.06012, -84.43733,  41.21004, -68.06012, -84.34557

AttributeError: 'list' object has no attribute 'shape'

<Figure size 1080x1080 with 0 Axes>

In [91]:
import os, sys, glob
import numpy as np
import dicom
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.draw import polygon

def read_structure(structure):
    contours = []
    for i in range(len(structure.ROIContourSequence)):
        contour = {}
        contour['color'] = structure.ROIContourSequence[i].ROIDisplayColor
        contour['number'] = structure.ROIContourSequence[i].RefdROINumber
        contour['name'] = structure.StructureSetROISequence[i].ROIName
        assert contour['number'] == structure.StructureSetROISequence[i].ROINumber
        contour['contours'] = [s.ContourData for s in structure.ROIContourSequence[i].ContourSequence]
        contours.append(contour)
    return contours

def get_mask(contours, slices):
    z = [s.ImagePositionPatient[2] for s in slices]
    pos_r = slices[0].ImagePositionPatient[1]
    spacing_r = slices[0].PixelSpacing[1]
    pos_c = slices[0].ImagePositionPatient[0]
    spacing_c = slices[0].PixelSpacing[0]

    label = np.zeros_like(image, dtype=np.uint8)
    for con in contours:
        num = int(con['number'])
    for c in con['contours']:
        nodes = np.array(c).reshape((-1, 3))
        assert np.amax(np.abs(np.diff(nodes[:, 2]))) == 0
        z_index = z.index(nodes[0, 2])
        r = (nodes[:, 1] - pos_r) / spacing_r
        c = (nodes[:, 0] - pos_c) / spacing_c
        rr, cc = polygon(r, c)
        label[rr, cc, z_index] = num

    colors = tuple(np.array([con['color'] for con in contours]) / 255.0)
    return label, colors

train_data_path = "./train"
train_patients = [os.path.join(train_data_path, name)
            for name in os.listdir(train_data_path) if os.path.isdir(os.path.join(train_data_path, name))]

patient = train_patients[0] # Just get the first patient for demo
for subdir, dirs, files in os.walk(patient):
    dcms = glob.glob(os.path.join(subdir, "*.dcm"))
    if len(dcms) == 1:
        structure = pydicom.read_file(os.path.join(subdir, files[0]))
        print({k:getattr(structure,k) for k in structure.__dir__()})
        contours = read_structure(structure)
    elif len(dcms) > 1:
        slices = [dicom.read_file(dcm) for dcm in dcms]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        image = np.stack([s.pixel_array for s in slices], axis=-1)
        label, colors = get_mask(contours, slices)

# Plot to check slices, for example 50 to 59
plt.figure(figsize=(15, 15))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image[..., i + 50], cmap="gray")
    plt.contour(label[..., i + 50], levels=[0.5, 1.5, 2.5, 3.5, 4.5], colors=colors)
    plt.axis('off')

AttributeError: 'FileDataset' object has no attribute 'PixelData'